In [3]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go
import plotly.express  as px

In [4]:
#DEMO 全場用電量data
site = pd.read_csv(r'datasets\ETT-data\TC3 POWER 15MIN.csv')
elc = pd.read_csv(r'datasets\ETT-data\TC3 TPC 15MIN.csv')
data = pd.merge(elc,site,how='inner',on='Dtime')
col = ['Dtime'] + [f'd{c}' for c in range(2,37)] + ['Power']
data = data[col]
data = data[data.Dtime > '2022-07'].reset_index(drop=True)
data = data.drop(data.std()[(data.std() == 0)].index, axis=1)
data.to_csv(r'datasets\ETT-data\TrainOneDay.csv',index=False)

C:\Users\ZME\AppData\Local\Temp\ipykernel_628\1270272191.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = data.drop(data.std()[(data.std() == 0)].index, axis=1)


In [5]:
#訓練使用csv 路徑為.\datasets\ETT-data\TrainOneDay.csv
#欄位 '日期' ,'(features1,2,3,4)' , '預測目標'

'''
df_raw.columns: ['date', ...(other features), target feature]
'''

data = pd.read_csv(r'datasets\ETT-data\TrainOneDay.csv')

#丟棄NA與std=0欄位
data = data.dropna(axis=0).drop(data.std()[(data.std() == 0)].index, axis=1)

#調整欄位名稱OT為目標欄位
data = data.rename(columns={
    data.columns[0]:'date',  data.columns[-1]:'OT'
})
data  = data[['date'] + list(data.columns[2:-1]) + ['OT']]

#儲存處裡完成csv
data.to_csv(r'datasets\ETT-data\ETTh1.csv',index=False)

C:\Users\ZME\AppData\Local\Temp\ipykernel_628\838334311.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = data.dropna(axis=0).drop(data.std()[(data.std() == 0)].index, axis=1)


In [6]:
#0.8訓練 0.1驗證 0.1測試
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.date,y=data.OT,line=dict(width=2)))
fig.add_vrect(
    x0=data.date.iloc[int(data.shape[0]*0.8)],
    x1=data.date.iloc[int(data.shape[0]*0.9)],
    fillcolor="red",
    opacity=0.2,
    line_width=0,
)

fig.add_vrect(
    x0=data.date.iloc[int(data.shape[0]*0.9)],
    x1=data.date.iloc[-1],
    fillcolor="green",
    opacity=0.2,
    line_width=0,
)

In [7]:

# 重要參數
# --features     type=str,  default='MS'     特徵選取 S:單變量預測單變量(y->y) MS:多變量預測單變量(x+y->y) M:多變量預測多變量(x+y->x+y)(未測試)
# --seq_len      type=int,  default=480      用多少筆資料進行預測 EX: 15min*192 = 2day 
# --label_len    type=int,  default=96       預測標籤長度 EX: 15min*96 = 1day 
# --pred_len     type=int,  default=96       預測長度    EX: 15min*96 = 1day 
# --model'       type=str,  default='SCINet' 模型紀錄名稱
# --model_name   type=str,  default='SCINet' Tensor board 上模型名稱 (啟動方式tensorboard --logdir .\event) http://localhost:6006/

# 輔助工具
# --evaluate     type=bool, default=False    進行Test data驗證(倒數10%) 結果儲存至 .\exp\ett_results
# --evaluateALL  type=bool, default=False    進行ALL data驗證 結果儲存至 .\exp\ett_results
# --infer        type=bool, default=False    落地實際預測 取.\dataset\infer.csv 倒數seq_len筆 進行預測

# 模型配置
# --train_epochs type=int,  default=100      訓練迭代次數
# --patience     type=int,  default=15       Valid set 幾次 loss 沒有下降提早終止訓練
# --hidden-size  type=int,  default=2        隱藏層數量 建議1~5
# --batch_size   type=int,  default=8        訓練批次大小 建議2,4,8,16,32,64,128,256
# --lr'          type=float default=3e-3     學習速率建議 1e-3 ~ 1e-5

In [10]:
!python .\run_ETTh.py --hidden-size 2 --batch_size 16  --seq_len 192 --label_len 96 --pred_len 96 --features MS --model_name h2_b16_2day_ms 


Args in experiment:
Namespace(model='SCINet', data='ETTh1', root_path='./datasets/ETT-data/', data_path='ETTh1.csv', features='MS', target='OT', freq='h', checkpoints='exp/ETT_checkpoints/', inverse=False, embed='timeF', use_gpu=True, gpu=0, use_multi_gpu=False, devices='0', seq_len=192, label_len=96, pred_len=96, concat_len=0, single_step=0, single_step_output_One=0, lastWeight=1.0, cols=None, num_workers=0, itr=0, train_epochs=100, batch_size=16, patience=15, lr=0.003, loss='rmse', lradj=1, use_amp=False, save=True, model_name='h2_b16_2day_ms', resume=False, evaluate=False, evaluateAll=False, infer=False, hidden_size=2.0, INN=1, kernel=5, dilation=1, window_size=12, dropout=0.5, positionalEcoding=False, groups=1, levels=3, stacks=1, num_decoder_layer=1, RIN=False, decompose=False, enc_in=34, dec_in=34, c_out=1, detail_freq='h')
SCINet(
  (blocks1): EncoderTree(
    (SCINet_Tree): SCINet_Tree(
      (workingblock): LevelSCINet(
        (interact): InteractorLevel(
          (level): I

In [9]:
from ipywidgets import interact
from glob import glob
import matplotlib.pyplot as plt
files = glob(r'exp\ett_results\*\*')

from sklearn.metrics import mean_squared_error
def rmse(true,pred):
    return mean_squared_error(true.flatten(),pred.mean(axis=2).flatten())**0.5

In [2]:
@interact
def load(f=files):
    global true,pred
    true = np.load(rf'{f}\true_scales.npy')
    pred = np.load(rf'{f}\pred_scales.npy')
    L = true.shape[0]
    print('TRAIN RMSE:{:.2f}'.format(rmse(true[0:int(L*0.8)],pred[0:int(L*0.8)])))
    print('VALID RMSE:{:.2f}'.format(rmse(true[int(L*0.8):int(L*0.9)],pred[int(L*0.8):int(L*0.9)])))
    print('TEST  RMSE:{:.2f}'.format(rmse(true[int(L*0.9):-1],pred[int(L*0.9):-1])))
    @interact
    def show(x=(0,true.shape[0]-1,1)):
        fig = plt.figure(figsize=(20,5))
        if x > true.shape[0]*0.8:
            plt.title('Valid')
            if x > true.shape[0]*0.9:
                plt.title('Test')
        else:
            plt.title('Train')
        plt.ylim([true.min(), true.max()])
        plt.plot(true[x].flatten())
        plt.plot(pred[x].mean(axis=1).flatten())
        plt.show()

interactive(children=(Dropdown(description='f', options=(), value=None), Output()), _dom_classes=('widget-inte…